#  迁移学习

In [1]:
import os
import shutil

#图片分类，创建软链接
train_filenames = os.listdir('imagedata/train')
valid_count=int(len(train_filenames)*0.1)

valid_data=train_filenames[-valid_count:]
train_data=train_filenames[:-valid_count]

train_cat = filter(lambda x:x[:3] == 'cat', train_data)
train_dog = filter(lambda x:x[:3] == 'dog', train_data)

valid_cat = filter(lambda x:x[:3] == 'cat', valid_data)
valid_dog = filter(lambda x:x[:3] == 'dog', valid_data)

def cre_rem_dir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cre_rem_dir('train')
os.mkdir('train/cat')
os.mkdir('train/dog')

cre_rem_dir('valid')
os.mkdir('valid/cat')
os.mkdir('valid/dog')


cur_path=os.getcwd()

for filename in train_cat:
    os.symlink(cur_path+'/imagedata/train/'+filename, 'train/cat/'+filename)
for filename in train_dog:
    os.symlink(cur_path+'/imagedata/train/'+filename, 'train/dog/'+filename)
for filename in valid_cat:
    os.symlink(cur_path+'/imagedata/train/'+filename, 'valid/cat/'+filename)
for filename in valid_dog:
    os.symlink(cur_path+'/imagedata/train/'+filename, 'valid/dog/'+filename)    

In [2]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.applications import *


train_datagen =  ImageDataGenerator(
    preprocessing_function=inception_v3.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(
    preprocessing_function=inception_v3.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


train_generator = train_datagen.flow_from_directory(
  'train',
  target_size=(299, 299),
  batch_size=16,
)
validation_generator = test_datagen.flow_from_directory(
  'valid',
  target_size=(299, 299),
  batch_size=16,
)

Using TensorFlow backend.


Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [5]:
base_model = InceptionV3(weights='imagenet', include_top=False)

#添加新层
def add_new_last_layer(base_model, nb_classes):
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x) 
    predictions = Dense(nb_classes, activation='softmax')(x) 
    model = Model(input=base_model.input, output=predictions)
    
    return model


def setup_to_transfer_learn(model, base_model):
    for layer in base_model.layers:
        layer.trainable = False
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy', 
                      metrics=['accuracy'])

        
model=add_new_last_layer(base_model,2)

setup_to_transfer_learn(model,base_model)

/Users/hl/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
history = model.fit_generator(
  train_generator,
  nb_epoch=5,
  validation_data=validation_generator,
  class_weight='auto')

model.save(args.output_model_file)

/Users/hl/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/Users/hl/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., class_weight="auto", epochs=5)`
  """


Epoch 1/5
1407/1407 [==============================] - 13450s 10s/step - loss: 0.2849 - acc: 0.9080 - val_loss: 0.0798 - val_acc: 0.9668
Epoch 2/5
 239/1407 [====>.........................] - ETA: 67:53:34 - loss: 0.1469 - acc: 0.9383

In [ ]:
history = model.fit_generator(
  train_generator,
  samples_per_epoch=nb_train_samples,
  nb_epoch=nb_epoch,
  validation_data=validation_generator,
  nb_val_samples=nb_val_samples,
  class_weight='auto')

model.save(args.output_model_file)









In [54]:
from sklearn import preprocessing
def one_hot(x):
    lb = preprocessing.LabelBinarizer()
    return lb.fit_transform(x)
class_list=one_hot(class_list)

#分离数据集
test_count=int(len(img_list)*0.1)
x_test = img_list[-test_count:]
y_test = class_list[-test_count:]

x_train=img_list[:-test_count]
y_train=class_list[:-test_count]


#转化为numpy.array
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)


In [55]:
lb = preprocessing.LabelBinarizer()
y_train=lb.fit_transform(y_train)
y_test=lb.fit_transform(y_test)


In [19]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

In [58]:
print(x_train.shape)
print(y_train)


(22500, 224, 224, 3)
[[1]
 [0]
 [1]
 ..., 
 [1]
 [1]
 [0]]


In [64]:
from keras.utils import to_categorical

# #数据预处理
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

# for e in range(epochs):
#     print 'Epoch', e
#     batches = 0
#     for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
#         loss = model.train(x_batch, y_batch)
#         batches += 1
#         if batches >= len(x_train) / 32:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#             break

AttributeError: 'numpy.ndarray' object has no attribute 'to_categorical'

In [60]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y=_test, batch_size=32)

ValueError: You are passing a target array of shape (22500, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

In [43]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(256, (3,3),activation='relu'))
model.add(Conv2D(256, (3,3),activation='relu'))
model.add(Conv2D(256, (3,3),activation='relu'))
model.add(Conv2D(256, (3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3,3),activation='relu'))
model.add(Conv2D(512, (3,3),activation='relu'))
model.add(Conv2D(512, (3,3),activation='relu'))
model.add(Conv2D(512, (3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3,3),activation='relu'))
model.add(Conv2D(512, (3,3),activation='relu'))
model.add(Conv2D(512, (3,3),activation='relu'))
model.add(Conv2D(512, (3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=32)

ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_148/convolution' (op: 'Conv2D') with input shapes: [?,1,1,512], [3,3,512,512].

In [40]:
#  Generate dummy data
x1_train = np.random.random((200, 100, 100, 3))
y1_train = keras.utils.to_categorical(np.random.randint(10, size=(200, 1)), num_classes=10)
x1_test = np.random.random((20, 100, 100, 3))
y1_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x1_train, y1_train, batch_size=32, epochs=10)
score = model.evaluate(x1_test, y1_test, batch_size=32)

Epoch 1/10
200/200 [==============================] - 14s 69ms/step - loss: 2.3461
Epoch 2/10
200/200 [==============================] - 12s 58ms/step - loss: 2.2929
Epoch 3/10
200/200 [==============================] - 12s 58ms/step - loss: 2.2994
Epoch 4/10
200/200 [==============================] - 14s 71ms/step - loss: 2.2918
Epoch 5/10
200/200 [==============================] - 11s 56ms/step - loss: 2.2988
Epoch 6/10
200/200 [==============================] - 11s 56ms/step - loss: 2.2848
Epoch 7/10
200/200 [==============================] - 11s 55ms/step - loss: 2.2804
Epoch 8/10
200/200 [==============================] - 11s 54ms/step - loss: 2.2822
Epoch 9/10
200/200 [==============================] - 11s 54ms/step - loss: 2.2849
Epoch 10/10
20/20 [==============================] - 1s 29ms/step


In [35]:
print (x1_train.shape)

(100, 100, 100, 3)


In [36]:
print(y1_train.shape)

(100, 10)
